In [1]:
!pip install faiss-cpu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=57c41c7a5fb5fd984237ed709413d6afea8233b00065601f10ed4a3654ce028e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-

In [2]:
import pandas as pd
data = [['Where are your headquarters located?', 'location'],
['Throw my cellphone in the water', 'random'],
['Network Access Control?', 'networking'],
['Address', 'location']]
df = pd.DataFrame(data, columns = ['text', 'category'])

In [3]:
df

,text,category
0,Where are your headquarters located?,location
1,Throw my cellphone in the water,random
2,Network Access Control?,networking
3,Address,location


In [4]:
from sentence_transformers import SentenceTransformer
text = df['text']
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
vectors = encoder.encode(text)

In [5]:
vectors

array([[-0.0043762 , -0.06387775, -0.11621021, ...,  0.09268747,
        -0.04782985, -0.05484065],
       [-0.01245353, -0.26505393, -0.04051866, ...,  0.14551327,
         0.13894185,  0.10772911],
       [-0.01477676,  0.05492531,  0.03377846, ...,  0.08851656,
        -0.14657764, -0.02152409],
       [-0.00393083,  0.12958978, -0.03537   , ...,  0.03184689,
        -0.11042541,  0.02578498]], dtype=float32)

In [6]:
vectors.shape

(4, 768)

In [7]:
import faiss

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

In [8]:
import numpy as np

search_text = 'where is your office?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

In [9]:
k = index.ntotal
distances, ann = index.search(_vector, k=k)

In [10]:
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

In [11]:
results

,distances,ann
0,0.584873,0
1,1.175950,3
2,1.644266,2
3,1.919768,1


In [13]:
labels  = df['category']
category = labels[ann[0][0]]
category, labels

('location',
 0      location
 1        random
 2    networking
 3      location
 Name: category, dtype: object)